In [1]:
!pip install -q bitsandbytes
!pip install -q transformers accelerate peft datasets sentencepiece safetensors


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 59.4/59.4 MB 20.5 MB/s eta 0:00:00


In [2]:
BASE_MODEL = "deepseek-ai/DeepSeek-R1-Distill-Llama-8B"
USE_4BIT = True

# Google Drive paths
from google.colab import drive
drive.mount('/content/drive')

PROJECT_DIR = "/content/drive/MyDrive/medical_finetuning"
ADAPTER_SAVE = PROJECT_DIR + "/adapter"
DATA_TRAIN = PROJECT_DIR + "/train.csv"
DATA_VALID = PROJECT_DIR + "/valid.csv"

import os
os.makedirs(ADAPTER_SAVE, exist_ok=True)


Mounted at /content/drive


In [10]:
import pandas as pd

data = {
    "instruction": [
        "What are the common symptoms of diabetes?",
        "How should a patient manage high blood pressure?",
        "Explain the first aid steps for a bone fracture.",
        "What is the treatment for migraine headaches?",
        "How to control high cholesterol naturally?"
    ],
    "output": [
        "Common symptoms include increased thirst, frequent urination, fatigue, blurred vision, and slow-healing wounds.",
        "Patients should reduce salt intake, exercise regularly, maintain a healthy weight, monitor blood pressure, and take prescribed medications.",
        "Immobilize the injured area, apply ice, prevent movement, and seek professional medical help immediately.",
        "Migraine treatment includes rest, pain relievers, hydration, avoiding triggers, and sometimes prescription medications.",
        "Consume fiber-rich foods, avoid fried items, exercise regularly, and increase omega-3 intake."
    ]
}

df = pd.DataFrame(data)
df.to_csv("/content/train.csv", index=False)

print("train.csv created successfully at /content/train.csv")
df


train.csv created successfully at /content/train.csv


,instruction,output
0,What are the common symptoms of diabetes?,"Common symptoms include increased thirst, freq..."
1,How should a patient manage high blood pressure?,"Patients should reduce salt intake, exercise r..."
2,Explain the first aid steps for a bone fracture.,"Immobilize the injured area, apply ice, preven..."
3,What is the treatment for migraine headaches?,"Migraine treatment includes rest, pain relieve..."
4,How to control high cholesterol naturally?,"Consume fiber-rich foods, avoid fried items, e..."


In [12]:
from datasets import load_dataset

dataset = load_dataset("csv", data_files={"train": "/content/train.csv"})
dataset


Generating train split: 0 examples [00:00, ? examples/s]

DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 5
    })
})

In [13]:
from datasets import load_dataset
dataset = load_dataset("csv", data_files={"train": "/content/train.csv"})
dataset


DatasetDict({
    train: Dataset({
        features: ['instruction', 'output'],
        num_rows: 5
    })
})

In [15]:
!pip install -q unsloth
!pip install -q bitsandbytes accelerate transformers datasets


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 61.8/61.8 kB 3.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 353.0/353.0 kB 15.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 506.8/506.8 kB 35.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 564.7/564.7 kB 49.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 283.5/283.5 kB 28.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 122.9/122.9 MB 7.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 132.6/132.6 kB 12.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 47.7/47.7 MB 18.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.2/7.2 MB 72.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 213.6/213.6 kB 18.7 MB/s eta 0:00:00


In [16]:
from unsloth import FastLanguageModel
import torch

max_seq_length = 2048
dtype = None
load_in_4bit = True

model, tokenizer = FastLanguageModel.from_pretrained(
    model_name = "unsloth/llama-3-8b-bnb-4bit",
    max_seq_length = max_seq_length,
    dtype = dtype,
    load_in_4bit = load_in_4bit,
)


🦥 Unsloth: Will patch your computer to enable 2x faster free finetuning.
🦥 Unsloth Zoo will now patch everything to make training faster!
==((====))==  Unsloth 2025.11.3: Fast Llama patching. Transformers: 4.57.1.
   \\   /|    Tesla T4. Num GPUs = 1. Max memory: 14.741 GB. Platform: Linux.
O^O/ \_/ \    Torch: 2.9.0+cu126. CUDA: 7.5. CUDA Toolkit: 12.6. Triton: 3.5.0
\        /    Bfloat16 = FALSE. FA [Xformers = 0.0.33.post1. FA2 = False]
 "-____-"     Free license: http://github.com/unslothai/unsloth
Unsloth: Fast downloading is enabled - ignore downloading bars which are red colored!


model.safetensors:   0%|          | 0.00/5.70G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/198 [00:00<?, ?B/s]

tokenizer_config.json: 0.00B [00:00, ?B/s]

tokenizer.json: 0.00B [00:00, ?B/s]

special_tokens_map.json:   0%|          | 0.00/350 [00:00<?, ?B/s]

In [17]:
from peft import prepare_model_for_kbit_training, LoraConfig, get_peft_model

model = prepare_model_for_kbit_training(model)

lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["q_proj", "k_proj", "v_proj", "o_proj"],
    lora_dropout=0.05,
    bias="none",
    task_type="CAUSAL_LM"
)

model = get_peft_model(model, lora_config)
model.print_trainable_parameters()


trainable params: 6,815,744 || all params: 8,037,076,992 || trainable%: 0.0848


In [19]:
print(dataset["train"].column_names)


['instruction', 'output']


In [20]:
def preprocess(example):
    text = f"Question: {example['prompt']}\nAnswer: {example['completion']}"
    tokens = tokenizer(text, truncation=True, max_length=MAX_LENGTH, padding="max_length")
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens


In [22]:
print(dataset["train"].column_names)


['instruction', 'output']


In [26]:
print(dataset.keys())


dict_keys(['train'])


In [28]:
# Split 80% train, 20% validation
split_dataset = dataset["train"].train_test_split(test_size=0.2)
train_ds = split_dataset["train"].map(preprocess)
val_ds   = split_dataset["test"].map(preprocess)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [31]:
MAX_LENGTH = 512

def preprocess(example):
    text = f"Question: {example['instruction']}\nAnswer: {example['output']}"
    tokens = tokenizer(text, truncation=True, max_length=MAX_LENGTH, padding="max_length")
    tokens["labels"] = tokens["input_ids"].copy()
    return tokens

# Split the training dataset into train and validation (80% train, 20% val)
split_dataset = dataset["train"].train_test_split(test_size=0.2)

# Apply preprocessing
train_ds = split_dataset["train"].map(preprocess)
val_ds   = split_dataset["test"].map(preprocess)


Map:   0%|          | 0/4 [00:00<?, ? examples/s]

Map:   0%|          | 0/1 [00:00<?, ? examples/s]

In [34]:
import transformers
print(transformers.__version__)


4.57.1


In [35]:
!pip install --upgrade transformers


In [41]:
ADAPTER_SAVE = "adapter_output"  # or any path you want


In [42]:
from transformers import TrainingArguments


In [47]:
!pip uninstall adapter-transformers -y
!pip install --upgrade transformers

In [45]:
from transformers import TrainingArguments as HFTrainingArguments

training_args = HFTrainingArguments(
    output_dir="adapter_output",
    per_device_train_batch_size=1,
    evaluation_strategy="steps",
    save_steps=100,
    logging_steps=10,
    num_train_epochs=3
)


TypeError: TrainingArguments.__init__() got an unexpected keyword argument 'evaluation_strategy'

In [49]:
import os
from transformers import TrainingArguments, Trainer

os.environ["WANDB_DISABLED"] = "true"

training_args = TrainingArguments(
    output_dir=ADAPTER_SAVE,
    per_device_train_batch_size=1,
    gradient_accumulation_steps=16,
    num_train_epochs=1,
    learning_rate=2e-4,
    fp16=True,
    logging_steps=20,
    save_total_limit=1,
    report_to="none" # Disable wandb logging
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=train_ds,
    eval_dataset=val_ds,
    tokenizer=tokenizer
)

trainer.train()

/tmp/ipython-input-3563651779.py:18: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `Trainer._unsloth___init__`. Use `processing_class` instead.
  trainer = Trainer(
==((====))==  Unsloth - 2x faster free finetuning | Num GPUs used = 1
   \\   /|    Num examples = 5 | Num Epochs = 1 | Total steps = 1
O^O/ \_/ \    Batch size per device = 1 | Gradient accumulation steps = 16
\        /    Data Parallel GPUs = 1 | Total batch size (1 x 16 x 1) = 16
 "-____-"     Trainable parameters = 6,815,744 of 8,037,076,992 (0.08% trained)


Unsloth: Will smartly offload gradients to save VRAM!


Step,Training Loss


TrainOutput(global_step=1, training_loss=169.24899291992188, metrics={'train_runtime': 31.3568, 'train_samples_per_second': 0.159, 'train_steps_per_second': 0.032, 'total_flos': 115380332789760.0, 'train_loss': 169.24899291992188, 'epoch': 1.0})

In [50]:
model.save_pretrained(ADAPTER_SAVE)
tokenizer.save_pretrained(ADAPTER_SAVE)

print("Adapter saved to:", ADAPTER_SAVE)


Adapter saved to: adapter_output


In [52]:
import torch
from peft import PeftModel
from transformers import AutoModelForCausalLM # Import AutoModelForCausalLM

# The base model and tokenizer are already loaded and configured by Unsloth
# using `model` and `tokenizer` variables from the previous cells.
# So, we can directly use the `model` and `tokenizer` for inference.

ft_model = PeftModel.from_pretrained(model, ADAPTER_SAVE)
ft_model.eval()

def ask(q):
    prompt = f"Question: {q}\nAnswer:"
    inputs = tokenizer(prompt, return_tensors="pt").to(ft_model.device)
    out = ft_model.generate(**inputs, max_new_tokens=200)
    print(tokenizer.decode(out[0], skip_special_tokens=True))

ask("What is the treatment for fever?")

/usr/local/lib/python3.12/dist-packages/peft/tuners/tuners_utils.py:282: UserWarning: Already found a `peft_config` attribute in the model. This will lead to having multiple adapters in the model. Make sure to know what you are doing!
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/peft/peft_model.py:598: UserWarning: Found missing adapter keys while loading the checkpoint: ['base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.q_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.k_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_A.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.v_proj.lora_B.default.weight', 'base_model.model.base_model.model.model.layers.0.self_attn.o_proj.

Question: What is the treatment for fever?
Answer: Fever is not a disease. It is a symptom of a disease. In a healthy person, the body temperature is maintained at 36.5°C to 37.5°C. If it increases above 37.5°C, it is called fever. Fever is a protective mechanism of the body to fight against infections. Fever is not to be treated. It is the disease that causes fever that is to be treated. In case of fever, the child should be given plenty of fluids to drink. In case of high fever, the child should be given sponge bath. The child should be given proper rest.
